In [1]:
# using ast.literal_eval()
import sys
import ast
import os
import time
import pandas as pd
import pickle
import glob
import numpy as np
import statistics
from scipy.stats import mannwhitneyu
import math

In [2]:
with open("../wgcna_tutorial/unweighted_pan_tissue_paths.7.dictionary", 'rb') as config_dictionary_file:
 
    # Step 3
    paths = pickle.load(config_dictionary_file)
    
pan_7_paths = []


for key,val in paths.items() :
    for keys,vals in val.items():
        pan_7_paths.append(vals)

statistics.mean(pan_7_paths)

6.779439125677435

In [3]:
with open("../wgcna_tutorial/unweighted_pan_tissue_paths.5.dictionary", 'rb') as config_dictionary_file:
 
    # Step 3
    paths = pickle.load(config_dictionary_file)
    
pan_5_paths = []


for key,val in paths.items() :
    for keys,vals in val.items():
        pan_5_paths.append(vals)

statistics.mean(pan_5_paths)

2.9592028675823574

In [4]:
with open("../wgcna_tutorial/unweighted_pan_tissue_paths.64.dictionary", 'rb') as config_dictionary_file:
 
    # Step 3
    paths = pickle.load(config_dictionary_file)
    
pan_64_paths = []


for key,val in paths.items() :
    for keys,vals in val.items():
        pan_64_paths.append(vals)

statistics.mean(pan_64_paths)

4.84132915889356

In [3]:
# create list
file2 = "common_nodes_hPPIN_STRING_PANCAN.txt"
combined_list = []
with open(file2) as IN:
    for line in IN:
        combined_list.append(line.rstrip().split("\t")[0])

In [4]:
def compute_sp(list1, list2, paths):
    total_sp = []
    for i in range(len(list1)):
        for j in range(len(list2)):
            if(list1[i] in paths.keys()):
                shortest_paths = paths[list1[i]]
                if(list2[j] in shortest_paths.keys()):
                    total_sp.append(shortest_paths[list2[j]])
    return({"distribution": total_sp,
            "mean": statistics.mean(total_sp),
            "median": statistics.median(total_sp)})

In [7]:
# Read Cancersea Data
folder_path = "/data/timonaj/cancer_as_wound/ppi_analysis/hallmark"

filenames = os.listdir(folder_path)

hallmark = {}

for filename in filenames:
    file_path = f"{folder_path}/{filename}"
    hallmark[filename[:-4]] = list()

    with open(file_path) as IN:
        for line in IN:
            hallmark[filename[:-4]].append(line.rstrip())

In [8]:
# read in HP sets
folder_path = "/data/timonaj/cancer_as_wound/ppi_analysis/HP_genesets/"

filenames = os.listdir(folder_path)

hp_genesets = {}

for filename in filenames:
    file_path = f"{folder_path}/{filename}"
    hp_genesets[filename[:-4]] = list()

    with open(file_path) as IN:
        for line in IN:
            hp_genesets[filename[:-4]].append(line.rstrip().split("\t")[0])

In [9]:
hallmark = hp_genesets

In [11]:
# read in HP matched controls set
folder_path = "/data/timonaj/cancer_as_wound/ppi_analysis/HP_genesets_matched_controls/"

filenames = glob.glob("/data/timonaj/cancer_as_wound/ppi_analysis/HP_genesets_matched_controls/pan_tissue*")
hp_genesets_matched = {}

for filename in filenames:
    name = filename.split("pan_tissue_")[1][:-4]
    hp_genesets_matched[name] = list()

    with open(filename) as IN:
        for line in IN:
            hp_genesets_matched[name].append(line.rstrip().split("\t")[0])

In [12]:
a_mean = np.zeros(shape=(len(hp_genesets), len(hallmark)))
a_median = np.zeros(shape=(len(hp_genesets), len(hallmark)))
a_dist = pd.DataFrame(index = range(len(hallmark)),
                      columns = range(len(hp_genesets)))

In [13]:
i=0
for key1,value1 in hp_genesets.items():
    j=0
    for key2,value2 in hallmark.items():
        current_info = compute_sp(value1, value2, paths)
        a_mean[i][j] = current_info["mean"]
        a_median[i][j] = current_info["median"]
        a_dist[i][j] = current_info["distribution"]
        j+=1
    i+=1

In [14]:
pd.DataFrame(a_median)

,0,1,2,3,4,5,6,7,8
0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
1,4.0,5.0,4.0,4.0,5.0,5.0,5.0,5.0,5.0
2,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,5.0
3,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,5.0
4,4.0,5.0,4.0,4.0,5.0,5.0,5.0,5.0,5.0
5,4.0,5.0,4.0,4.0,5.0,4.0,4.0,5.0,5.0
6,4.0,5.0,4.0,4.0,5.0,4.0,4.0,5.0,5.0
7,4.0,5.0,4.0,4.0,5.0,5.0,5.0,5.0,5.0
8,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0


In [15]:
pd.DataFrame(a_mean)

,0,1,2,3,4,5,6,7,8
0,4.160579,4.471044,4.342918,4.288661,4.520404,4.370015,4.416489,4.476259,4.547712
1,4.471044,4.587512,4.548130,4.534247,4.682839,4.602336,4.610866,4.628957,4.648913
2,4.342918,4.548130,4.402113,4.401792,4.566566,4.505500,4.520582,4.566697,4.620567
3,4.288661,4.534247,4.401792,4.331136,4.561484,4.468967,4.492138,4.563504,4.613844
4,4.520404,4.682839,4.566566,4.561484,4.647753,4.671776,4.671075,4.702582,4.744805
5,4.370015,4.602336,4.505500,4.468967,4.671776,4.513254,4.569934,4.630538,4.670220
6,4.416489,4.610866,4.520582,4.492138,4.671075,4.569934,4.571566,4.622831,4.656799
7,4.476259,4.628957,4.566697,4.563504,4.702582,4.630538,4.622831,4.594313,4.668202
8,4.547712,4.648913,4.620567,4.613844,4.744805,4.670220,4.656799,4.668202,4.645328


In [16]:
pd.DataFrame(a_mean, columns = hallmark.keys(), index = hp_genesets.keys()).to_csv("pan_tissue_meanSP_HP_Phenotypes_hallmark_9by9.csv")

In [17]:
a_matched_mean = np.zeros(shape=(len(hp_genesets_matched), len(hallmark)))
a_matched_median = np.zeros(shape=(len(hp_genesets_matched), len(hallmark)))
a_matched_dist = pd.DataFrame(index = range(len(hallmark)),
                              columns = range(len(hp_genesets_matched)))

In [18]:
i=0
for key1,value1 in hp_genesets_matched.items():
    j=0
    for key2,value2 in hallmark.items():
        current_info = compute_sp(value1, value2, paths)
        a_matched_mean[i][j] = current_info["mean"]
        a_matched_median[i][j] = current_info["median"]
        a_matched_dist[i][j] = current_info["distribution"]
        j+=1
    i+=1

In [19]:
pd.DataFrame(a_matched_median)

,0,1,2,3,4,5,6,7,8
0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
1,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
2,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
3,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
4,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
5,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
6,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
7,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
8,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0


In [20]:
pd.DataFrame(a_matched_mean)

,0,1,2,3,4,5,6,7,8
0,4.653240,4.746021,4.722262,4.727669,4.842613,4.782249,4.770040,4.742765,4.774067
1,4.609797,4.698217,4.673954,4.686413,4.790804,4.746230,4.727322,4.686289,4.729359
2,4.737266,4.832293,4.830009,4.829273,4.948987,4.865108,4.858011,4.823999,4.853979
3,4.643011,4.710975,4.699240,4.718433,4.810968,4.768927,4.744386,4.688752,4.731759
4,4.738012,4.827089,4.808334,4.816313,4.926844,4.867555,4.855798,4.825559,4.860323
5,4.628615,4.729733,4.703653,4.708433,4.827971,4.759280,4.749903,4.721019,4.757127
6,4.578040,4.664204,4.652608,4.655016,4.763984,4.706742,4.690582,4.656136,4.688648
7,4.672305,4.748514,4.735301,4.742576,4.843697,4.794463,4.773733,4.739167,4.769127
8,4.780395,4.851888,4.837921,4.854998,4.944066,4.902500,4.882091,4.857839,4.879427


In [21]:
pd.DataFrame(a_matched_mean, columns = hallmark.keys(), index = hp_genesets_matched.keys()).to_csv("pan_tissue_meanSP_HPmatched_Phenotypes_hallmark_9by9.csv")

In [22]:
mannwhitneyu_p = np.zeros(shape=(len(hp_genesets), len(hallmark)))
test_dist = np.zeros(shape=(len(hp_genesets), len(hallmark)))
test_matched_dist = np.zeros(shape=(len(hp_genesets), len(hallmark)))

In [23]:
for i in range(len(hp_genesets)):
    for j in range(len(hallmark)):
        actual_dist = a_dist[i][j]
        control_dist = a_matched_dist[i][j]
        res = mannwhitneyu(actual_dist, control_dist, alternative="less")
        mannwhitneyu_p[i][j] = res.pvalue
        test_dist[i][j] = statistics.mean(actual_dist)
        test_matched_dist[i][j] = statistics.mean(control_dist)

In [24]:
hallmark.keys()

dict_keys(['downregulated.wound', 'upregulated.stress', 'downregulated.regen', 'msigdb.wound', 'msigdb.regen', 'upregulated.regen', 'upregulated.wound', 'msigdb.stress', 'downregulated.stress'])

In [25]:
pvals = pd.DataFrame(mannwhitneyu_p, columns = hallmark.keys(), index = hp_genesets.keys())

In [26]:
pvals

,downregulated.wound,upregulated.stress,downregulated.regen,msigdb.wound,msigdb.regen,upregulated.regen,upregulated.wound,msigdb.stress,downregulated.stress
downregulated.wound,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.054537e-240,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
upregulated.stress,8.783888e-253,4.584533e-77,7.550880e-148,5.458380e-152,1.750770e-22,3.459356e-211,7.223784e-133,2.228841e-30,6.523953e-36
downregulated.regen,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.293438e-210,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
msigdb.wound,0.000000e+00,1.725305e-266,0.000000e+00,0.000000e+00,2.240092e-118,0.000000e+00,0.000000e+00,1.912384e-285,5.331897e-117
msigdb.regen,0.000000e+00,1.011964e-127,0.000000e+00,1.065340e-232,2.258211e-73,1.409569e-254,2.861074e-245,1.701439e-225,4.245125e-87
upregulated.regen,0.000000e+00,4.983665e-188,0.000000e+00,0.000000e+00,2.726142e-66,0.000000e+00,0.000000e+00,2.717992e-216,9.783360e-94
upregulated.wound,0.000000e+00,6.284943e-42,5.376123e-215,1.649251e-195,1.057174e-28,5.076576e-223,7.809732e-168,6.214168e-37,5.343523e-19
msigdb.stress,0.000000e+00,2.093967e-150,0.000000e+00,7.017459e-229,2.869195e-52,0.000000e+00,1.824613e-285,0.000000e+00,1.614821e-101
downregulated.stress,1.296676e-245,5.670451e-126,1.741424e-178,1.038597e-126,2.107963e-32,6.182239e-186,2.252643e-190,8.056518e-248,9.329006e-148


In [27]:
pvals.to_csv("pan_tissue_matched_pvals_hallmark_9by9.csv")

In [28]:
LogFC = np.zeros(shape=(len(hp_genesets), len(hallmark)))
for i in range(len(hp_genesets)):
    for j in range(len(hallmark)):
        test = test_dist[i][j]
        matched =test_matched_dist[i][j]
        LogFC[i][j] = math.log((test/matched))

In [29]:
LFC = pd.DataFrame(LogFC, columns = hallmark.keys(), index = hp_genesets.keys())

In [30]:
LFC

,downregulated.wound,upregulated.stress,downregulated.regen,msigdb.wound,msigdb.regen,upregulated.regen,upregulated.wound,msigdb.stress,downregulated.stress
downregulated.wound,-0.111910,-0.059685,-0.083741,-0.097458,-0.068853,-0.090144,-0.077010,-0.057833,-0.048574
upregulated.stress,-0.030562,-0.023845,-0.027289,-0.033009,-0.022794,-0.030787,-0.024943,-0.012309,-0.017156
downregulated.regen,-0.086914,-0.060605,-0.092764,-0.092684,-0.080421,-0.076790,-0.071988,-0.054813,-0.049281
msigdb.wound,-0.079388,-0.038236,-0.065389,-0.085647,-0.053250,-0.064964,-0.054633,-0.027076,-0.025236
msigdb.regen,-0.047016,-0.030339,-0.051589,-0.054361,-0.058315,-0.041052,-0.038784,-0.025815,-0.024055
upregulated.regen,-0.057491,-0.027305,-0.043040,-0.052198,-0.032887,-0.053078,-0.038625,-0.019352,-0.018438
upregulated.wound,-0.035926,-0.011501,-0.028787,-0.035617,-0.019695,-0.029497,-0.025701,-0.007179,-0.006816
msigdb.stress,-0.042865,-0.025500,-0.036255,-0.038490,-0.029567,-0.034789,-0.032121,-0.031042,-0.021389
downregulated.stress,-0.049899,-0.042735,-0.045968,-0.050947,-0.041138,-0.048539,-0.047245,-0.039820,-0.049166


In [31]:
LFC.to_csv("pan_tissue_matched_LFC_hallmark_9by9.csv")

In [32]:
pd.DataFrame(test_dist)

,0,1,2,3,4,5,6,7,8
0,4.160579,4.471044,4.342918,4.288661,4.520404,4.370015,4.416489,4.476259,4.547712
1,4.471044,4.587512,4.548130,4.534247,4.682839,4.602336,4.610866,4.628957,4.648913
2,4.342918,4.548130,4.402113,4.401792,4.566566,4.505500,4.520582,4.566697,4.620567
3,4.288661,4.534247,4.401792,4.331136,4.561484,4.468967,4.492138,4.563504,4.613844
4,4.520404,4.682839,4.566566,4.561484,4.647753,4.671776,4.671075,4.702582,4.744805
5,4.370015,4.602336,4.505500,4.468967,4.671776,4.513254,4.569934,4.630538,4.670220
6,4.416489,4.610866,4.520582,4.492138,4.671075,4.569934,4.571566,4.622831,4.656799
7,4.476259,4.628957,4.566697,4.563504,4.702582,4.630538,4.622831,4.594313,4.668202
8,4.547712,4.648913,4.620567,4.613844,4.744805,4.670220,4.656799,4.668202,4.645328


In [33]:
pd.DataFrame(test_matched_dist)

,0,1,2,3,4,5,6,7,8
0,4.653240,4.746021,4.722262,4.727669,4.842613,4.782249,4.770040,4.742765,4.774067
1,4.609797,4.698217,4.673954,4.686413,4.790804,4.746230,4.727322,4.686289,4.729359
2,4.737266,4.832293,4.830009,4.829273,4.948987,4.865108,4.858011,4.823999,4.853979
3,4.643011,4.710975,4.699240,4.718433,4.810968,4.768927,4.744386,4.688752,4.731759
4,4.738012,4.827089,4.808334,4.816313,4.926844,4.867555,4.855798,4.825559,4.860323
5,4.628615,4.729733,4.703653,4.708433,4.827971,4.759280,4.749903,4.721019,4.757127
6,4.578040,4.664204,4.652608,4.655016,4.763984,4.706742,4.690582,4.656136,4.688648
7,4.672305,4.748514,4.735301,4.742576,4.843697,4.794463,4.773733,4.739167,4.769127
8,4.780395,4.851888,4.837921,4.854998,4.944066,4.902500,4.882091,4.857839,4.879427
